# Neo-davidsonian composition
## Author: Kyle Rawlins

This notebook demonstrates an implementation of a basic neo-Davidsonian treatment of semantic composition.

In [ ]:
from IPython.display import display
#reload_lamb() # may be useful if you edit this notebook

The classic account of verb meaning is that it a verb relates individuals.  A transitive verb is a two-place relation in type e.

In [ ]:
type_e = types.type_e
type_t = types.type_t
type_n = types.type_n

In [ ]:
%%lamb
||smokes|| = L x_e : Smokes(x)
||sees|| = L y_e : L x_e : Sees(x, y)
||alfonso|| = Alfonso_e
||joanna|| = Joanna_e

In [ ]:
smokes * alfonso

In [ ]:
alfonso * (sees * joanna)

In [ ]:
(alfonso * (sees * joanna)).tree()

There are many things that are tricky on this classical account.  The best argument is probably from adverbs.  I will not repeat the arguments in detail here, but the most basic one is what is sometimes called the 'diamond' entailment pattern (see e.g. Parsons 1990).  (1) illustrates the basic pattern, and (2)-(5) illustrate the diamond pattern.

(1) Alfonso danced slowly $\leadsto$ Alfonso danced

(2) Alfonso danced slowly in the kitchen.

(3) $\leadsto$ Alfonso danced slowly.

(4) $\leadsto$ Alfonso danced in the kitchen.

(5) $\leadsto$ Alfonso danced.

In general, core arguments do not show this sort of pattern, and it is not straightforward to handle under the relation account.  The pattern in (2)-(5) in fact resembles what happens with conjunction, which led Davidson to propose a somewhat different logical form for sentences like (2):

(6) $\exists e : Dancing(e, Alfonso) \wedge Slow(e) \wedge In(e, kitchen)$

$e$ here is an event variable, and on Davidson's original account, verbs denote a relation on events and event participants.  In the neo-Davidsonian account, developed below, even participant combination is separated out.

First we add a type for events ($v$):

In [ ]:
type_v = types.BasicType("v")
ts = meta.get_type_system()
ts.add_atomic(type_v)
ts

On this neo-Davidsonian treatment, VPs denote properties of events.  Predicates such as $Agent$ and $Patient$ in the meta-language relate events with participants.  Composition with a subject is mediated by a Kratzer-inspired little v, and composition with a direct object happens classically (though still mediated by a thematic predicate).

To keep things simple, I have done everything here via Function Application.

In [ ]:
%%lamb
agent = Agent_<(v,e),t> # this is a way of enforcing consistent types for this predicate.
patient = Patient_<(v,e),t>
||smokes|| = L e_v : Smoking(e)
||sees|| = L y_e : L e_v : Seeing(e) & patient(e,y)
||ACTIVE|| = L f_<v,t> : L x_e : L e_v : f(e) & agent(e,x)

In [ ]:
v = ACTIVE
r = (alfonso * (v * smokes))
r

In [ ]:
r2 = (alfonso * (v * (sees * joanna)))
r2

In [ ]:
r2.tree()

Despite the complicated sequence of composition, the end result is just a conjunction of event properties (more or less).

Now, adverbs have a simple treatment, analogous to adjectives:

In [ ]:
%%lamb
||dance|| = L e_v : Dancing(e)
||slowly|| = L f_<v,t> : L e_v : f(e) & Slow(e)
||inP|| = L x_e : L f_<v,t> : L e_v : f(e) & Loc(e, x)
||the|| = L f_<e,t> : Iota x_e : f(x)
||kitchen|| = L x_e : Kitchen(x)

In [ ]:
itk = inP * (the * kitchen)
itk

In [ ]:
dance0 = (alfonso * (v * dance))
dance1 = (alfonso * (v * (dance * slowly)))
dance2 = (alfonso * (v * (dance * itk)))
dance3 = (alfonso * (v * ((dance * slowly) * itk)))
display(dance0, dance1, dance2, dance3)

In [ ]:
dance3.tree()

The denotations above remain unsaturated.  Part of Davidson's idea is that the truth-conditions of an "action sentence" involve existentially quantifying over an event.  There are a number of proposals as to how these existential quantifier is introduced compositionally, but the simplest is to treat it as a type shift (or unary composition operation).

We can first do this by defining a function that performs "existential closure", and then defining a composition operation using that function.

In [ ]:
ec_fun = te("L f_<v,t> : Exists e_v : f(e)")
ec_fun

In [ ]:
(ec_fun(dance3[0].content)).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_unary_rule(ec_fun, "EC")
lang.set_system(system)
system

Unary composition can be triggered by composing with python `None`.

In [ ]:
dance4 = (dance3 * None)
dance4

In [ ]:
dance4.tree()

The above treatment uses Function Application for every stage.  The alternative is to use a (generalized) version of Predicate Modification, or some similar operation (e.g. Chung and Ladusaw's Restrict).

The version of PM provided by the lambda notebook library is specific to type $\langle e,t \rangle$, so we will need to generalize.  The library implemention of PM uses the function shown below as `basic_pm_fun`, and to generalize, one can construct such an operator for arbitrary input types.  The function `construct_gpm_fun` does this, with two example property types shown.

In [ ]:
basic_pm_fun = lang.te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")
basic_pm_fun

In [ ]:
gpm_fun = %te L f_<X,t> : L g_<X,t> : L x_X : f(x) & g(x)
gpm_fun

In [ ]:
system = lang.td_system.copy()
system.add_unary_rule(ec_fun, "EC")
system.add_binary_rule(gpm_fun, "GPM", commutative=True)
lang.set_system(system)
system

In [ ]:
(dance * slowly).tree()

Note that PM is redundant with GPM, so we may want to remove PM:

In [ ]:
system.remove_rule("PM")
system

Now with those preliminaries out of the way, we are in good shape to reformulate the neo-Davidsonian account using mainly GPM.

Just to demonstrate what this could involve, I'll alter the assumption about little-v.  The early implementation assumed that the subject composed in Spec,TP; now we can even assume it is in Spec,vP.  This leads to much simpler entries all around, and is close to Kratzer's original proposal.

In [ ]:
%%lamb
# redo everything
||smokes|| = L e_v : Smoking(e)
||sees|| = L y_e : L e_v : Seeing(e) & patient(e,y)
||ACTIVE|| = L x_e : L e_v : agent(e,x)
||dance|| = L e_v : Dancing(e)
||slowly|| = L e_v : Slow(e)
||inP|| = L x_e : L e_v : Loc(e, x)
||the|| = L f_<e,t> : Iota x_e : f(x)
||kitchen|| = L x_e : Kitchen(x)

In [ ]:
v = ACTIVE
((alfonso * v) * smokes).tree()

In [ ]:
((alfonso * v) * ((dance * slowly) * (inP * (the * kitchen)))) * None

In [ ]:
(((alfonso * v) * ((dance * slowly) * (inP * (the * kitchen)))) * None).tree()

Of course, this raises many questions that are not answered by the basic compositional architecture.  What is an agent/patient?  What does it mean for an event to be `Slow`?  Etc.  However, the basic desiderata of the diamond entailment now follow, assuming any reasonable treatment of $\wedge$; see Parsons, Landman, and many many others for development of the Davidsonian program.

In the remainder of the notebook I discuss the conjunction of action sentences, such as _Alfonso talked quickly and Alfonso danced slowly_.

In [ ]:
%%lamb
||quickly|| = L e_v : Quick(e)
||talk|| = L e_v : Talking(e)
||and_|| = L p_t : L q_t : p & q

In [ ]:
((alfonso * v) * (dance * quickly)) * None

In [ ]:
((alfonso * v) * (talk * slowly)) * None

In [ ]:
((((alfonso * v) * (dance * quickly)) * None)) * (and_ * (((alfonso * v) * (talk * slowly)) * None))

Does this work for all cases of conjunction? The current analysis would a wrong result (though it does get a result) for something like: _Alfonso talked slowly and danced quickly_ unless the syntax is treated as conjunction of full sentences at LF.

If you want to handle VP conjunction without LF-based assumptions, you basically need to get existential closure to somehow scope under the subject.  Three strategies for doing so:
1. Introduce existential closure in v itself, and have one v per conjunct.  This requires a higher-typed version of `and`.
2. Introduce existential closure inside the lexical items (proposed by e.g. Lucas Champollion).  This leads to complications for the modifiers (which could be solved by e.g. an existential disclosure operation).
3. A very different strategy would be to have a different type for conjunction that interacts directly with the event structure.  The idea would be to build a property of plural events that has a part that is a slow talking, and a part that is a quick dancing.  Then Alfonso would be the agent of the plural event; you'd want to be able to infer also that he's the agent of the parts.

Here's a version of solution 1.

In [ ]:
%%lamb 
||ACTIVE2|| = L f_<v,t> : L x_e : Exists e_v : agent(e,x) & f(e)
||and2|| = L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)

In [ ]:
r = alfonso * ((ACTIVE2 * (dance * quickly)) * (and2 * (ACTIVE2 * (talk * slowly))))
r

In [ ]:
r.tree()